<a href="https://colab.research.google.com/github/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355_201714378/blob/main/Trabajo_Final_Hito_3_Elena_Salcedo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ALGORITMO ELENA SALCEDO

##IMPORTS

In [1]:
import graphviz as gv
import numpy as np
import pandas as pd
import heapq as hq
import time
import math

In [2]:
url = "https://raw.githubusercontent.com/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355_201714378/main/DATASET/almacenesXyY.csv"
url2 = "https://raw.githubusercontent.com/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355_201714378/main/DATASET/puntosentregaXyY.csv"
file_almacenes = pd.read_csv(url).to_numpy()
pts_entrega = pd.read_csv(url2).to_numpy()

##ELABORACIÓN

###LA_MATRIX

In [7]:
la_matrix = []
for i in range(1000):
    l = [0]*1000                          # N°0 -> ni almacén ni punto de entrega
    la_matrix.append(l)

for line in file_almacenes:
    la_matrix[line[0]][line[1]] = 1       # N°1 -> almacenes

for line in pts_entrega:
    if la_matrix[line[0]][line[1]] != 1:
       la_matrix[line[0]][line[1]] = 2    # N°2 -> puntos de entrega

In [8]:
def mostrar_matriz():
  for line in la_matrix:
    print(line)

###PUNTOS

In [9]:
puntos = list()
n = 1000
for i in range(n*n):
    l = [0]*4
    puntos.append(l)

for i in range(n):
  for j in range(n):
    puntos[j+n*i][0] = j                # 0 = eje x
    puntos[j+n*i][1] = i                # 1 = eje y
    puntos[j+n*i][2] = la_matrix[i][j]  # 2 = tipo de punto
    puntos[j+n*i][3] = -1               # 3 = padre

###GRAFO

In [10]:
grafo = list()

for i in range(n):
  for j in range(n):
    l = list()
    if j+n*i-1000 >= 0:
      l.append(j+n*i-1000)
    if j+n*i-1 >= 0:
      l.append(j+n*i-1)
    if j+n*i+1 <= 999999:
      l.append(j+n*i+1)
    if j+n*i+1000 <= 999999:
      l.append(j+n*i+1000)
    
    grafo.append(l)

###NEAREST NEIGHBOR

In [11]:
pts_almacenes = []
pts_entregar = []

In [12]:
for i in range(n*n):
  if puntos[i][2] == 1:
    pts_almacenes.append(i)
  elif puntos[i][2] == 2:
    pts_entregar.append(i)

In [13]:
def clusters(warehouse_points, delivery_points): # La funcion clusters pone los puntos de entrega al punto de almacenes más cercano  
  manhattan = lambda a,b: abs(puntos[a][0]-puntos[b][0])+abs(puntos[a][1]-puntos[b][1])
  cl = []
  for i in range(len(warehouse_points)):
    cl.append([])

  for j in range(len(delivery_points)):
    d = delivery_points[j]
    mn = math.inf
    idmn = 0
    for i in range(len(warehouse_points)):
      w = warehouse_points[i]
      dis = manhattan(d,w) #se obtiene la distancia de cada warehouse_points hacia las los delivery_points
      if(dis < mn):
        mn = dis
        idmn = i
    
    cl[idmn].append(d)
  
  return cl

In [14]:
clusters_ = clusters(pts_almacenes,pts_entregar)

In [15]:
def give(clst,orden_pnt_almacen):
  new = pts_almacenes[0]
  for i in range(len(clst[orden_pnt_almacen])):
    minima_distancia = math.inf
    for punto_entrega in clst[orden_pnt_almacen]:
      if puntos[punto_entrega][3]==-1:
        distancia = abs(puntos[punto_entrega][0]-puntos[new][0])+abs(puntos[punto_entrega][1]-puntos[new][1])

        if distancia < minima_distancia:
          minima_distancia = distancia
          ultimo_punto_entrega = punto_entrega

    puntos[ultimo_punto_entrega][3] = new
    new = ultimo_punto_entrega

In [16]:
for cluster in range(len(clusters_)):
  give(clusters_,cluster)

###RESULTADO FINAL

In [17]:
print("PUNTO\tX   Y TIPO PADRE")
for point in pts_entregar:
  print(point,":",puntos[point])

PUNTO	X   Y TIPO PADRE
24 : [24, 0, 2, 8687]
397 : [397, 0, 2, 77368]
918 : [918, 0, 2, 18904]
1246 : [246, 1, 2, 14248]
1430 : [430, 1, 2, 10432]
2316 : [316, 2, 2, 9318]
2420 : [420, 2, 2, 397]
2716 : [716, 2, 2, 9713]
2936 : [936, 2, 2, 918]
3197 : [197, 3, 2, 20193]
3619 : [619, 3, 2, 4603]
3706 : [706, 3, 2, 4721]
4066 : [66, 4, 2, 10067]
4337 : [337, 4, 2, 6335]
4603 : [603, 4, 2, 5598]
4678 : [678, 4, 2, 8687]
4721 : [721, 4, 2, 2716]
5104 : [104, 5, 2, 6123]
5580 : [580, 5, 2, 6563]
5598 : [598, 5, 2, 5580]
6123 : [123, 6, 2, 10128]
6221 : [221, 6, 2, 3197]
6335 : [335, 6, 2, 19335]
6487 : [487, 6, 2, 8488]
6563 : [563, 6, 2, 18523]
8488 : [488, 8, 2, 57461]
8855 : [855, 8, 2, 8687]
8964 : [964, 8, 2, 77948]
9318 : [318, 9, 2, 8687]
9364 : [364, 9, 2, 22367]
9663 : [663, 9, 2, 9670]
9670 : [670, 9, 2, 4678]
9713 : [713, 9, 2, 17712]
9823 : [823, 9, 2, 20826]
10059 : [59, 10, 2, 4066]
10067 : [67, 10, 2, 8687]
10097 : [97, 10, 2, 5104]
10128 : [128, 10, 2, 26124]
10296 : [296, 1

###DATOS EXTRA

In [19]:
cantidades = []
for cluster in clusters_:
  cantidades.append(len(cluster))

In [20]:
from statistics import mean
from statistics import median
from statistics import mode

In [23]:
print("MAX:",max(cantidades))
print("MIN:",min(cantidades))
print("MEDIA:",mean(cantidades))
print("MEDIANA:",median(cantidades))
print("MODA:",mode(cantidades))

MAX: 123
MIN: 4
MEDIA: 34.95
MEDIANA: 31.0
MODA: 29
